In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from statistics import mean
from datetime import timedelta

import math

import warnings  
warnings.filterwarnings('ignore')

In [101]:
data = pd.read_csv("~/Desktop/NCSA_genomics/Python - notebooks/GlucoCheck/Data/consolidatedDataForPaper.csv")
data['Display Time'] = pd.to_datetime(data['Display Time'])
data['GlucoseValue'] = pd.to_numeric(data['GlucoseValue'])

In [102]:
def fullDay(data):
        
    dates = list()
    data = data.reset_index(drop=True)
    for i in range(0,len(data.index)):
        dates.append(data['Display Time'][i].date())
    data['Dates'] = dates
    end = data['Dates'].iloc[-1]
    start = data['Dates'].iloc[0]

    indexVals = data[ data['Dates'] == start ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    indexVals = data[ data['Dates'] == end ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    data = data.reset_index(drop=True)

    data.drop(['Dates'], axis=1, inplace=True)

    return data

In [103]:
def congaN(df, n):
    day = df['Display Time'].iloc[-1]-df['Display Time'].iloc[0]
    day = day.round("d")
    day = day.days
    
    df = df.set_index(['Display Time'])
    t = str(n*3600)+'s'
    gv = df['GlucoseValue'].resample(t).first()
    
    k = len(gv)
    
    frame = pd.DataFrame()
    frame['GV'] = gv
    frame['Dt'] = frame['GV'] - frame['GV'].shift(+1)
    frame = frame.fillna(0)
    
    dBar = sum(frame['Dt']) / k
    
    s = 0
    for i in frame['Dt']:
        s += (i-dBar)**2
        
    conga = math.sqrt(s/(k-1))
    
    return round(conga/day, 3)
    
    
    

In [104]:
for subjectId, df in data.groupby('subjectId'):
    df = fullDay(df)
    conga_1 = congaN(df, 1)
    conga_2 = congaN(df, 2)
    conga_4 = congaN(df, 4)
    print("{}\t{}\t{}\t{}".format(subjectId,conga_1,conga_2,conga_4))

GVP01	9.103	15.098	24.613
GVP03	0.071	0.096	0.116
ID01	9.036	14.448	16.64
ID02	3.065	3.028	3.942
ID03	5.72	7.817	11.583
ID11	9.717	14.784	25.069
ID12	4.675	8.115	9.069
ID13	13.095	20.574	12.434
ID21	6.111	8.892	6.467
ID22	7.553	10.99	9.945
ID23	4.56	6.433	7.697
ID29	7.07	10.241	8.492
ID30	4.327	5.266	3.896
ID31	5.293	6.34	5.77
ID32	1.885	1.894	1.87
ID33	1.159	1.364	1.074
OD540	0.944	1.349	1.69
OD544	0.879	1.36	1.471
OD552	0.944	1.403	1.72
OD559	1.213	1.813	2.356
OD563	0.76	1.168	1.415
OD567	0.909	1.397	1.708
OD570	0.784	1.264	1.836
OD575	0.958	1.462	1.792
OD584	0.984	1.446	1.748
OD588	0.831	1.209	1.557
OD591	0.93	1.306	1.784
OD596	0.649	1.012	1.406
